### ML-7. Прогнозирование биологического ответа (HW-3)
В поле ниже введите ссылку на выполненное задание. 

КРИТЕРИИ ОЦЕНКИ

Балл|Критерий
-|-
0|Задание не выполнено
1|Обучено две модели; гипепараметры подобраны при помощи одного метода
2|Обучено две модели; гипепараметры подобраны при помощи двух методов
3|Обучено две модели; гипепараметры подобраны при помощи трёх методов
4|Обучено две модели; гипепараметры подобраны при помощи четырёх методов
5|Обучено две модели; гипепараметры подобраны при помощи четырёх методов; использована кросс-валидация

In [1]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
#import matplotlib.pyplot as plt #для визуализации
#import seaborn as sns #для визуализации
import warnings
warnings.filterwarnings("ignore")

from sklearn import linear_model #линейные моделиё
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
#from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials

import optuna

In [2]:
# загрузим датасет
url = 'https://lms-cdn.skillfactory.ru/assets/courseware/v1/9f2add5bca59f8c4df927432d605fff3/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/_train_sem09__1_.zip'
data = pd.read_csv(url)

In [3]:
data.head()

,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


Создаем матрицу наблюдений $X$ и вектор ответов $y$

In [4]:
X = data.drop(columns=['Activity'])
y = data['Activity']

Разделяем выборку на тренировочную и тестовую. 

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 42, test_size = 0.2)

Обучим две модели: логистическую регрессию и случайный лес.

In [6]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(max_iter = 1000)
#Обучаем модель
log_reg.fit(X_train, y_train)
#Выводим значения метрики 
y_train_pred = log_reg.predict(X_train)
print('Train: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = log_reg.predict(X_test)
print('Test: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Train: 0.89
Test: 0.78


In [7]:
#Создаем объект класса дерево решений
dt = tree.DecisionTreeClassifier(random_state=42)
#Обучаем дерево по алгоритму CARTrint
dt.fit(X_train, y_train)
#Выводим значения метрики 
y_train_pred = dt.predict(X_train)
print('Train: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))
y_test_pred = dt.predict(X_test)
print('Test: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Train: 1.00
Test: 0.75


### Оптимизация гиперпараметров

### GridSearchCV

Логистическая регрессия

In [8]:
param_grid = {'penalty': ['l2', None] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]} # уровень силы регурялизации

grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=42,
        max_iter=2000
    ), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search.fit(X_train, y_train) 

y_test_pred = grid_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

CPU times: total: 11.6 s
Wall time: 16min 56s
f1_score на тестовом наборе: 0.78
Наилучшие значения гиперпараметров: {'C': 0.01, 'penalty': 'l2', 'solver': 'lbfgs'}


Улучшить метрику не получилось, она осталась прежней

Случайный лес

In [9]:
param_grid = {'min_samples_leaf': list(np.linspace(5, 100, 50, dtype=int)),
              'max_depth': list(np.linspace(1, 30, 30, dtype=int)),
              'criterion':['entropy','gini']
              }
            
grid_search_tree = GridSearchCV(
    estimator=tree.DecisionTreeClassifier(random_state=42), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search_tree.fit(X_train, y_train) 

y_test_pred = grid_search_tree.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_tree.best_params_))

CPU times: total: 1min 33s
Wall time: 32min 1s
f1_score на тестовом наборе: 0.77
Наилучшие значения гиперпараметров: {'criterion': 'gini', 'max_depth': 4, 'min_samples_leaf': 6}


Метрика немного улучшилась по сравнению с базовым решением.

### **RandomizedSearchCV**

Логистическая регрессия

In [10]:
param_distributions = {'penalty': ['l2', None] ,
              'solver': ['lbfgs', 'sag'],
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
            
random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=2000), 
    param_distributions=param_distributions, 
    cv=5, 
    n_iter = 28,
    n_jobs = -1
)  
%time random_search.fit(X_train, y_train) 

y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

CPU times: total: 5.09 s
Wall time: 18min 23s
f1_score на тестовом наборе: 0.78
Наилучшие значения гиперпараметров: {'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.01}


Улучшить метрику по прежнему не удалось, она осталась прежней

Случайный лес

In [11]:
param_distributions = {'min_samples_leaf': list(np.linspace(5, 100, 50, dtype=int)),
              'max_depth': list(np.linspace(1, 30, 30, dtype=int)),
              'criterion':['entropy','gini']
              }
            
random_search_tree = RandomizedSearchCV(
    estimator=tree.DecisionTreeClassifier(random_state=42), 
    param_distributions=param_distributions, 
    cv=5,
    n_iter = 50, 
    n_jobs = -1
)  
%time random_search_tree.fit(X_train, y_train) 

y_test_pred = random_search_tree.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search_tree.best_params_))

CPU times: total: 2.17 s
Wall time: 39.4 s
f1_score на тестовом наборе: 0.77
Наилучшие значения гиперпараметров: {'min_samples_leaf': 70, 'max_depth': 7, 'criterion': 'gini'}


По сравненю с базовым решением метрика немного улучшилась

### Hyperopt

Логистическая регрессия

In [12]:
# зададим пространство поиска гиперпараметров
penalty = ['l2', None]
solver = ['lbfgs', 'sag']
C = [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]
space = {'penalty': hp.choice('penalty', penalty),
         'solver': hp.choice('solver', solver),
         'C': hp.choice('C', C)
         }

# зафксируем random_state
random_state = 42

def hyperopt_lr(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'penalty': params['penalty'], 
              'solver': params['solver'], 
             'C': params['C']
              }

    # используем эту комбинацию для построения модели
    model = linear_model.LogisticRegression(**params, random_state=random_state, max_iter=2000)
  
    # обучаем модель
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

# начинаем подбор гиперпараметров
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_lr, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))


100%|██████████| 20/20 [13:50<00:00, 41.50s/trial, best loss: -0.7872758838461376]
Наилучшие значения гиперпараметров {'C': 0, 'penalty': 0, 'solver': 1}


In [13]:
# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(
    penalty=penalty[best['penalty']], 
    solver=solver[best['solver']],
    C=C[best['C']],
    random_state=random_state, 
    max_iter=2000
    )

model.fit(X_train, y_train)

y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.78


Оптимизируя гиперпараметры с помощью Hyperopt, удалось улучшить метрику до 0.78

Случайный лес

In [14]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 26, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }

# зафксируем random_state
random_state = 42

def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

# начинаем подбор гиперпараметров
trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.default_rng(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print('f1_score на обучающем наборе: {:.2f}'.format(metrics.f1_score(y_train, y_train_pred)))

y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

  0%|          | 0/20 [00:00<?, ?trial/s, best loss=?]

100%|██████████| 20/20 [01:26<00:00,  4.33s/trial, best loss: -0.8160803811393121]
Наилучшие значения гиперпараметров {'max_depth': 18.0, 'min_samples_leaf': 2.0, 'n_estimators': 103.0}
f1_score на обучающем наборе: 0.99
f1_score на тестовом наборе: 0.81


Оптимизируя гиперпараметры с помощью Hyperopt, удалось улучшить метрику до 0.81.

## Optuna

Логистическая регрессия

In [15]:
ls_penalty = ['l2', None]
ls_solver = ['lbfgs', 'sag']
ls_C = [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]
  
def optuna_lr(trial):
  # задаем пространства поиска гиперпараметров

  penalty = trial.suggest_categorical('penalty', ls_penalty)
  solver = trial.suggest_categorical('solver', ls_solver)
  C = trial.suggest_categorical('C', ls_C)
  
  # создаем модель
  model = linear_model.LogisticRegression(penalty=penalty,
                                          solver=solver,
                                          C=C,
                                          random_state=random_state, 
                                          max_iter=2000)
  
  # обучаем модель
  score = cross_val_score(model, X, y, cv=5, scoring="f1", n_jobs=-1).mean()
  
  return score

# cоздаем объект исследования
study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_lr, n_trials=20)

[I 2023-11-30 22:45:07,807] A new study created in memory with name: LogisticRegression


[I 2023-11-30 22:45:23,024] Trial 0 finished with value: 0.777110650025232 and parameters: {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.9}. Best is trial 0 with value: 0.777110650025232.
[I 2023-11-30 22:48:08,413] Trial 1 finished with value: 0.7571940378262074 and parameters: {'penalty': None, 'solver': 'sag', 'C': 0.5}. Best is trial 0 with value: 0.777110650025232.
[I 2023-11-30 22:48:45,108] Trial 2 finished with value: 0.7763478036221179 and parameters: {'penalty': 'l2', 'solver': 'sag', 'C': 0.9}. Best is trial 0 with value: 0.777110650025232.
[I 2023-11-30 22:51:37,072] Trial 3 finished with value: 0.7571940378262074 and parameters: {'penalty': None, 'solver': 'sag', 'C': 0.01}. Best is trial 0 with value: 0.777110650025232.
[I 2023-11-30 22:54:31,150] Trial 4 finished with value: 0.7571940378262074 and parameters: {'penalty': None, 'solver': 'sag', 'C': 0.3}. Best is trial 0 with value: 0.777110650025232.
[I 2023-11-30 22:54:33,882] Trial 5 finished with value: 0.78310489874445

In [16]:
# выводим результаты
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))

# рассчитаем точность для тестовой выборки
model = linear_model.LogisticRegression(**study.best_params, random_state=random_state, max_iter=2000)
model.fit(X_train, y_train)

y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Наилучшие значения гиперпараметров {'penalty': 'l2', 'solver': 'lbfgs', 'C': 0.1}
f1_score на обучающем наборе: 0.79
f1_score на тестовом наборе: 0.79


Оптимизируя гиперпараметры с помощью Optuna, удалось улучшить метрику до 0.79

Случайный лес

In [17]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 10, 30, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  
  # обучаем модель
  score = cross_val_score(model, X, y, cv=5, scoring="f1", n_jobs=-1).mean()

  return score

# cоздаем объект исследования
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)

[I 2023-11-30 23:00:58,646] A new study created in memory with name: RandomForestClassifier


[I 2023-11-30 23:01:02,670] Trial 0 finished with value: 0.7956347799133443 and parameters: {'n_estimators': 112, 'max_depth': 17, 'min_samples_leaf': 9}. Best is trial 0 with value: 0.7956347799133443.
[I 2023-11-30 23:01:06,943] Trial 1 finished with value: 0.8076710481714091 and parameters: {'n_estimators': 111, 'max_depth': 20, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.8076710481714091.
[I 2023-11-30 23:01:12,658] Trial 2 finished with value: 0.7986296913828852 and parameters: {'n_estimators': 174, 'max_depth': 13, 'min_samples_leaf': 8}. Best is trial 1 with value: 0.8076710481714091.
[I 2023-11-30 23:01:16,355] Trial 3 finished with value: 0.8011480668959127 and parameters: {'n_estimators': 107, 'max_depth': 27, 'min_samples_leaf': 8}. Best is trial 1 with value: 0.8076710481714091.
[I 2023-11-30 23:01:21,163] Trial 4 finished with value: 0.8065650471635983 and parameters: {'n_estimators': 111, 'max_depth': 28, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.807

In [18]:
# выводим результаты
print("Наилучшие значения гиперпараметров {}".format(study.best_params))
print("f1_score на обучающем наборе: {:.2f}".format(study.best_value))

# рассчитаем точность для тестовой выборки
model = ensemble.RandomForestClassifier(**study.best_params, random_state=random_state)
model.fit(X_train, y_train)

y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

Наилучшие значения гиперпараметров {'n_estimators': 200, 'max_depth': 22, 'min_samples_leaf': 2}
f1_score на обучающем наборе: 0.82
f1_score на тестовом наборе: 0.80


Оптимизируя гиперпараметры с помощью Optuna, удалось улучшить метрику до 0.80.